# Tile Server Process for Aurora Outputs

In [1]:
import os
import sys

sys.path.append('../')

print(os.getcwd())
print(os.listdir())

import yaml
from PIL import Image
import numpy as np
import torch
import xarray as xr
import matplotlib.pyplot as plt
import cfgrib
from datetime import datetime, timedelta
import time
from subprocess import Popen
from io import BytesIO
import ast
from cartopy import crs as ccrs, feature as cfeature
from datetime import datetime
from tile_server import save_tile, tile_into_folders
from serve import serve

/home/jgib124/aurora/aurora/afwerx
['serve.py', 'tile_server_process.ipynb', '__pycache__', 'aurora_regridding_process.ipynb', 'visualize.py', 'ssd1', 'flask_server.py', 'configs', 'tile_server.py', 'key.pem', 'tiles', 'cert.pem', 'aurora_inference_simple.py']


/home/jgib124/miniconda3/envs/aurora_test/lib/python3.11/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


Wasabi: 
	Endpoint:  https://s3.wasabisys.com/
	Bucket:  aa-archiver
IDrive: 
	Endpoint:  https://l5l2.va.idrivee2-56.com
	Bucket:  archive-mosaics


In [2]:
# Read in regridded data
ds = xr.open_dataset("tiles/aurora_predictions_regridded.nc", engine='netcdf4')

ds.variables.keys()

KeysView(Frozen({'surf_2t': <xarray.Variable (batch: 1, history: 1, y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'surf_10u': <xarray.Variable (batch: 1, history: 1, y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'surf_10v': <xarray.Variable (batch: 1, history: 1, y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'surf_msl': <xarray.Variable (batch: 1, history: 1, y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'static_z': <xarray.Variable (y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'static_slt': <xarray.Variable (y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'static_lsm': <xarray.Variable (y: 4608, x: 7168)> Size: 132MB
[33030144 values with dtype=float32], 'atmos_t': <xarray.Variable (batch: 1, history: 1, level: 13, y: 4608, x: 7168)> Size: 2GB
[429391872 values with dtype=float32], 'atmos_u': <xarray.Variable (batch: 1, history: 1, level: 13, y: 4608

In [ ]:
# NOTE: either 1D or 3D image
# TODO: check for number of channels and repeat if 1D
serve_dir = "/ssd1/aurora_maptuple/serve/v2"
noun = "notebook_test"
timestamp = "202301010000"
serve_port = '8000'
zooms = (4, 11)

for var in ['atmos_t', 'atmos_q', 'atmos_v', 'atmos_u']:
    selected_array = ds[var].isel(batch=0, level=0, history=0).to_numpy()
    tile_into_folders(serve_dir=serve_dir, noun=var, timestamp=timestamp, image=selected_array, zoom=zooms)

for var in ['surf_msl']:
    selected_array = ds[var].isel(batch=0, history=0).to_numpy()
    tile_into_folders(serve_dir=serve_dir, noun=var, timestamp=timestamp, image=selected_array, zoom=zooms)

## Serve generated tiles to an endpoint
- HTTP-based server
- Flask server

In [ ]:
# Generate an SSL Certificate to use temporarily
#  openssl req -x509 -days 1 -newkey rsa:2048 -keyout key.pem -out cert.pem

In [ ]:
# HTTP Only Approach
serve_dir = "/ssd1/aurora_maptuple/serve"
serve_port = "8888"
serve_address = "100.99.120.135"
cert_path = "cert.pem"
key_path = "key.pem"

proc = Popen('python serve.py --serve_dir "{}" --serve_port {} --serve_address "{}" --certfile "{}" --keyfile "{}"'.format(serve_dir, serve_port, serve_address, cert_path, key_path), shell=True)
print(time.ctime(), 'Serving directory "{}" at port {}'.format(serve_dir, serve_port))
proc.pid

In [ ]:
# https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=1,1&startTime=20230101T000000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/atmos_q/
# https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=1,1&startTime=20230101T000000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/atmos_v/
# https://maptuple.mrsv.co/?lat=39.833&lon=-98.585&zoom=7&frames=1,1&startTime=20230101T000000Z&split=mirror&base=https://map.acmeaom.com/dark/&radar=https://100.99.120.135:8888/v2/surf_msl/

In [ ]:
Popen('netstat -tulpn | grep {}'.format(serve_port), shell=True)

In [ ]:
Popen("kill {}".format(proc.pid), shell=True)

In [ ]:
%%sh 
kill 589971

### Flask Server

In [ ]:
import cv2
from flask_cors import CORS
from flask import Flask, send_file
import io

TILE_SIZE = 256
ZOOM = 7

CONUS_Y_SIZE = 4608
CONUS_X_SIZE = 7168

# Aurora size: 720, 1440
# CONUS Size: 4608, 7168

minTileX = 0
minTileY = 0
maxTileX = CONUS_X_SIZE // TILE_SIZE
maxTileY = CONUS_Y_SIZE // TILE_SIZE

minX = minTileX * TILE_SIZE
minY = minTileY * TILE_SIZE
maxX = maxTileX * TILE_SIZE
maxY = maxTileY * TILE_SIZE

var_serve_dr = "atmos_t"

application = Flask(__name__)
CORS(application) # Prevents CORS Errors

In [ ]:
def serve_jpg(image):
    '''Input RGB image and return servable jpg.'''
    output = io.BytesIO()
    image.save(output, format='jpeg')
    output.seek(0, 0)
    return send_file(output, mimetype='image/jpeg')

def serve_png(image):
    '''Input RGB image and return servable png.'''
    output = io.BytesIO()
    image.save(output, format='png')
    output.seek(0, 0)
    return send_file(output, mimetype='image/png')

In [ ]:
def resample(tile, zoom, x, y):
    pass

In [ ]:
@application.route('/v2/ref_d_syn/<YYYYMMDDHHMM>/<zoom>/<x>/<y>.png')
def get_png(YYYYMMDDHHMM, zoom, y, x):
    '''
    Process PNG Tile Requests, checking if tile is available or needs to be resampled

    Parameters
    ----------
    YYYYMMDDHHMM : str
        YYYYMMDDHHMM timestamp of the image.
    zoom : str
        Zoom level of the tile.
    y : str
        Y coordinate of the tile.
    x : str
        X coordinate of the tile.
    '''
    # NOTE: ignoring the date for now, this will come later
    tname = f"{serve_dir}/{zoom}/{x}/{y}.png"
    if os.path.isfile(tname):
        return send_file(tname)
    else:
        # Needs to get resampled to required zoom level
        # Will exist in zoom 7
        tname = f"{serve_dir}/7/{x}/{y}.png"
        if os.path.isfile(tname):
            tile = cv2.imread(tname)
            resampled = resample(tile, zoom, x, y)
            return serve_png(resampled)
        else:
            raise FileNotFoundError(f"Tile does not exist in Zoom 7: {tname}")

### Code for finding updates

In [ ]:
def get_updates(host : str, back_steps : int = 0, interval : int = 360, set_now : datetime = None):
    '''
    Returns the most current URL if data is present in the current or previous hour. 
    Will also return the hex UTC timestamp for provided URL

    Parameters
    ----------
    host : str
        Hostname of the server, e.g. MRMS, GFS, forecasts, etc.
    back_steps : int
        Timesteps to go back into time.
    interval : int = 360
        Timestep interval between updates in minutes
    set_now : datetime.datetime = None
        Set a specific datetime to check from. Defaults to None to use datetime.utcnow(), else
        accepts a datetime object of the format datetime(year, month, day, hour, minute, second)
    '''
    if set_now is None:
        now = datetime.utcnow()
    else:
        now = set_now

    # UTC for the nearest interval
    now = now - timedelta(minutes=(back_steps*interval + now.minute % interval),
                                    seconds=now.second, microseconds=now.microsecond)
    
    since_1970 = now - datetime(1970, 1, 1)
    
    timestamp = hex(int(since_1970.total_seconds()))

    print("Hex Timestamp: ", timestamp)
    print("Now Timestamp: ", now)
    print("Time since 1970: ", since_1970)


get_updates("fake_url", set_now=datetime(2023, 1, 1, 7))